In [85]:
from src.core.translate import Translator
import pandas as pd
from datasets import Dataset
import configparser
import json
from pathlib import Path
import re
from datetime import datetime
from src.utils.schemas import ResultSchema, ResultSchemaRationales

In [2]:
config = configparser.ConfigParser()
config.read('configs/conf.yaml')
cfg = config['DEFAULT']

In [3]:
system_message = Path(cfg['prompt_path']).open().read()
model_config = json.load(Path(cfg['model_config_path']).open())
example_data = json.load(Path(cfg['filepath_examples']).open())

intermediate_path = f'int_path_{model_config["model"]}_{datetime.now()}.json'
batch_size = int(cfg['batch_size'])
batch_result_dir = cfg['batch_result_dir']
batch_dir = cfg['batches']

## Translating

In [4]:
data = pd.read_csv(cfg['data_path'])
dataset = {colname: data[colname].tolist()
			for colname in cfg['cols'].split()}
input_dataset = Dataset.from_dict(dataset)

In [5]:
tr = Translator(system_message, model_config, example_data, batch_size, batch_result_dir, batch_dir)
translation_result = tr.translate(input_dataset, ResultSchema)

Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 745.12ba/s]


Processing batch 1/8...


Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 200.92ba/s]


Processing batch 2/8...


Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 476.84ba/s]


...

Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 565.57ba/s]


Processing batch 7/8...


Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 489.19ba/s]


Processing batch 8/8...


Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 615.81ba/s]


In [6]:
regex_check_enhlsih = re.compile(r'\b[a-zA-Z]{2,}\b')

In [7]:
def contains_english(text: str) -> bool:
	"""
	Checks if the given text contains English words.

	Args:
		text (str): Text to check.

	Returns:
		bool:		True if English words are found, False otherwise.
	"""
	return bool(re.search(regex_check_enhlsih, text))

In [8]:
texts_containing_english = [text for text in translation_result if contains_english(text['text_rus'])]

In [9]:
texts_containing_english

[{'id': '5',
  'text': "Thats totally normal, most people get that from time to time. It will pass, the when is the real question. Good news is you can work on the when, at least it works for me. I think the key is to accept whatever you are feeling, not fight it and not try to change it, even try to get yourself to cry. The point is really to change your mindset from 'i have depression i can't cure it, it is killing me' to 'i'm sad again for no reason, human brain is a pile of shit and buggy as hell, I'm gonny cry all day to give it what it wants'..",
  'text_rus': "Это totally нормально, большинство людей испытывают это от случая к случаю. Это пройдёт, но ключевой вопрос — когда. Хорошая новость в том, что ты можешь работать над этим 'когда', по крайней мере, это работает на мне. Я думаю, что ключ в том, чтобы принять то, что ты чувствуешь, не сопротивляться этому и не пытаться изменить это, даже попытаться дать себе возможность поплакать. Точка в том, чтобы изменить свое отношение с

In [10]:
for t in texts_containing_english:
    translation_result.remove(t)

### Correcting translation

In [11]:
config = configparser.ConfigParser()
config.read('configs/conf_correction.yaml')
cfg = config['DEFAULT']

In [12]:
system_message = Path(cfg['prompt_path']).open().read()
model_config = json.load(Path(cfg['model_config_path']).open())
example_data = json.load(Path(cfg['filepath_examples']).open())

batch_size = int(cfg['batch_size'])
batch_result_dir = cfg['batch_result_dir']
batch_dir = cfg['batches']

In [13]:
dataset_correction = {'id': [], 'text': []}
for item in texts_containing_english:
    dataset_correction['id'].append(item['id'])
    dataset_correction['text'].append(item['text_rus'])

In [14]:
input_dataset_correction = Dataset.from_dict(dataset_correction)

In [15]:
tr = Translator(system_message, model_config, example_data, batch_size, batch_result_dir, batch_dir)
translation_result_corrected = tr.translate(input_dataset_correction, ResultSchema)

Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 661.04ba/s]


Processing batch 1/1...


Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 331.25ba/s]


In [16]:
translation_result_corrected

[{'id': '5',
  'text': "Это totally нормально, большинство людей испытывают это от случая к случаю. Это пройдёт, но ключевой вопрос — когда. Хорошая новость в том, что ты можешь работать над этим 'когда', по крайней мере, это работает на мне. Я думаю, что ключ в том, чтобы принять то, что ты чувствуешь, не сопротивляться этому и не пытаться изменить это, даже попытаться дать себе возможность поплакать. Точка в том, чтобы изменить свое отношение с 'у меня депрессия, я не могу её вылечить, она убивает меня' на 'опять грустно без причины, человеческий мозг — куча дерьма и битый, я буду плакать весь день, чтобы дать ему, что он хочет'.",
  'text_rus': "Это полностью нормально, большинство людей испытывают это от случая к случаю. Это пройдёт, но ключевой вопрос — когда. Хорошая новость в том, что ты можешь работать над этим 'когда', по крайней мере, это работает на мне. Я думаю, что ключ в том, чтобы принять то, что ты чувствуешь, не сопротивляться этому и не пытаться изменить это, даже поп

In [17]:
print([contains_english(t['text_rus']) for t in translation_result_corrected])

[False, False, False]


In [18]:
for item in translation_result_corrected:
    if not contains_english(item['text_rus']):
        translation_result.append(item)

In [19]:
if intermediate_path:
	d = Dataset.from_list(translation_result)
	with open(intermediate_path, "w", encoding="utf-8") as f:
		json.dump(d.to_list(), f, ensure_ascii=False, indent=4)

## Translating rationales

In [39]:
config = configparser.ConfigParser()
config.read('configs/conf_rationales.yaml')
cfg = config['DEFAULT']

In [42]:
system_message = Path(cfg['prompt_path']).open().read()
model_config = json.load(Path(cfg['model_config_path']).open())
example_data = json.load(Path(cfg['filepath_examples']).open())
intermediate_path = f"data/int_path_{model_config['model']}_{datetime.now()}-rationales.json"

batch_size = int(cfg['batch_size'])
batch_result_dir = cfg['batch_result_dir']
batch_dir = cfg['batches']

In [43]:
with open(cfg['data_path']) as f:
    dataset_rationales = json.load(f)

In [44]:
dataset_rationales['id'] = [i for i in range(len(dataset_rationales['text_eng']))]

In [45]:
input_dataset_rationales = Dataset.from_dict(dataset_rationales)

In [46]:
tr = Translator(system_message, model_config, example_data, batch_size, batch_result_dir, batch_dir)
translation_result_rationales = tr.translate(input_dataset_rationales, ResultSchemaRationales)

Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 1097.99ba/s]


Processing batch 1/8...


Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 54.12ba/s]


Processing batch 2/8...


Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 373.29ba/s]


...

Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 506.86ba/s]


Processing batch 7/8...


Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 604.98ba/s]


Processing batch 8/8...


Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 512.31ba/s]


### Correcting rationales

In [47]:
texts_containing_english = [text for text in translation_result_rationales if contains_english(text['rationales_rus'])]

In [75]:
texts_containing_english

[{'id': '0',
  'text_rus': 'Это действительно так плохо? Возможно, это было умным решением, потому что вам было нужно это время, чтобы восстановиться. Вы проявляете к себе доброту, когда это необходимо, и это важно. Надеюсь, вы скоро почувствуете себя лучше.',
  'text_eng': "Is that really so bad? Maybe it was the smart decision because you needed that time to read recover. You're being kind to yourself when you need it and that's important. Hope you feel better soon.",
  'rationales_eng': 'Hope you feel better soon|',
  'rationales_rus': 'Hope you скоро почувствуете себя лучше.'}]

In [ ]:
for t in texts_containing_english:
    translation_result_rationales.remove(t)

In [88]:
config = configparser.ConfigParser()
config.read('configs/conf_correction.yaml')
cfg = config['DEFAULT']

In [89]:
system_message = Path(cfg['prompt_path']).open().read()
model_config = json.load(Path(cfg['model_config_path']).open())
example_data = json.load(Path(cfg['filepath_examples']).open())

batch_size = int(cfg['batch_size'])
batch_result_dir = cfg['batch_result_dir']
batch_dir = cfg['batches']

In [90]:
dataset_correction = {'id': [], 'text_rus': [], 'text_eng': [], 'rationales_eng': []}
for item in texts_containing_english:
    dataset_correction['id'].append(item['id'])
    dataset_correction['text_rus'].append(item['text_rus'])
    dataset_correction['text_eng'].append(item['text_eng'])
    dataset_correction['rationales_eng'].append(item['rationales_rus'])

In [91]:
input_dataset_correction = Dataset.from_dict(dataset_correction)

In [92]:
tr = Translator(system_message, model_config, example_data, batch_size, batch_result_dir, batch_dir)
translation_result_corrected = tr.translate(input_dataset_correction, ResultSchemaRationales)

Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 217.50ba/s]


Processing batch 1/1...


Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 356.78ba/s]


In [93]:
translation_result_corrected

[{'id': '0',
  'text_rus': 'Это действительно так плохо? Возможно, это было умным решением, потому что вам было нужно это время, чтобы восстановиться. Вы проявляете к себе доброту, когда это необходимо, и это важно. Надеюсь, вы скоро почувствуете себя лучше.',
  'text_eng': "Is that really so bad? Maybe it was the smart decision because you needed that time to read recover. You're being kind to yourself when you need it and that's important. Hope you feel better soon.",
  'rationales_eng': 'Hope you скоро почувствуете себя лучше.',
  'rationales_rus': 'Надеюсь, вы скоро почувствуете себя лучше.'}]

In [94]:
print([contains_english(t['text_rus']) for t in translation_result_corrected])

[False]


In [95]:
for item, rationale in zip(translation_result_corrected, texts_containing_english):
    if not contains_english(item['text_rus']):
        translation_result_rationales.append({'id': item['id'], 'rationales_eng': rationale['rationales_eng'], 'rationales_rus': item['text_rus'], 'text_rus': rationale['text_rus'], 'text_eng': rationale['text_eng']})

In [96]:
if intermediate_path:
	d = Dataset.from_list(translation_result_rationales)
	with open(intermediate_path, "w", encoding="utf-8") as f:
		json.dump(d.to_list(), f, ensure_ascii=False, indent=4)

In [97]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer('sentence-transformers/LaBSE')

In [98]:
for item in translation_result:
	embedding_rus = model.encode(item['text_rus'])
	embedding_eng = model.encode(item['text'])

	l1_diff = np.sum(np.abs(embedding_rus - embedding_eng))

	print(f"L1-норма (разница между векторами): {l1_diff}")

L1-норма (разница между векторами): 11.428401947021484
L1-норма (разница между векторами): 12.030906677246094
L1-норма (разница между векторами): 9.96324348449707
L1-норма (разница между векторами): 9.975038528442383
L1-норма (разница между векторами): 10.744906425476074
L1-норма (разница между векторами): 10.01419448852539
L1-норма (разница между векторами): 11.401578903198242
L1-норма (разница между векторами): 10.81946849822998
L1-норма (разница между векторами): 11.075176239013672
L1-норма (разница между векторами): 13.749931335449219
L1-норма (разница между векторами): 9.930123329162598
L1-норма (разница между векторами): 15.17106819152832
L1-норма (разница между векторами): 8.267402648925781
L1-норма (разница между векторами): 8.864448547363281
L1-норма (разница между векторами): 11.623388290405273
L1-норма (разница между векторами): 13.545312881469727
L1-норма (разница между векторами): 11.646284103393555
L1-норма (разница между векторами): 6.259558200836182
L1-норма (разница ме